# 01 - Classificação com Validação Cruzada Estratificada

Este notebook realiza o pré-processamento, separação de atributos/rótulos e avaliação de três algoritmos supervisionados:
- Árvore de Decisão (Random Forest)
- K-NN (K-Nearest Neighbors)
- MLP (Multi-Layer Perceptron)

A avaliação utiliza F1-score e matriz de confusão via validação cruzada estratificada k-fold, com `random_state` fixo para reprodutibilidade.

Observação: Ajuste o caminho do arquivo da base de dados na seção de carregamento.


In [ ]:
# Configurações gerais
RANDOM_STATE = 42
N_JOBS = -1
N_SPLITS = 5  # k-fold

# Exibição de opções do pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


## Carregamento de dados

Substitua `data/arquivo.csv` pelo caminho real. O rótulo (target) deve estar em uma coluna, por exemplo `target`. Se a base tiver valores ausentes, faremos tratamento na etapa de pré-processamento.


In [ ]:
# Leitura da base
# ATENÇÃO: ajuste o caminho abaixo para o arquivo correto
csv_path = '../data/arquivo.csv'
df = pd.read_csv(csv_path)
print(df.shape)
df.head()


## Pré-processamento

- Identificação de colunas numéricas e categóricas
- Tratamento de valores ausentes
- Padronização de numéricas e One-Hot em categóricas
- Separação de `X` e `y`


In [ ]:
# Inferir colunas numéricas e categóricas automaticamente (exclui a coluna alvo)
TARGET_COL = 'target'  # ajuste o nome aqui

# Checa se a coluna alvo existe
assert TARGET_COL in df.columns, f"Coluna alvo '{TARGET_COL}' não encontrada no DataFrame."

feature_cols = [c for c in df.columns if c != TARGET_COL]

dtypes = df[feature_cols].dtypes
numeric_features = dtypes[dtypes.kind in 'iufc'].index.tolist()
categorical_features = [c for c in feature_cols if c not in numeric_features]

print('Numéricas:', numeric_features)
print('Categóricas:', categorical_features)

X = df[feature_cols].copy()
y = df[TARGET_COL].copy()


In [ ]:
# Transformações: imputação + escala/one-hot
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=True, with_std=True))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop'
)

# Validação cruzada estratificada k-fold
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
scorer = {'f1_macro': make_scorer(f1_score, average='macro')}

print(preprocess)


## Modelagem e Avaliação (CV Estratificada)

Treinaremos e avaliaremos:
- Random Forest
- KNN
- MLP

Usaremos F1 macro e exibiremos a matriz de confusão a partir de `cross_val_predict` para cada modelo.


In [ ]:
def avaliar_modelo(nome, estimator):
    pipe = Pipeline(steps=[('preprocess', preprocess), ('model', estimator)])
    # cross_validate para f1 macro
    cv_results = cross_validate(
        pipe, X, y, cv=cv, scoring=scorer, n_jobs=N_JOBS, return_estimator=False, error_score='raise'
    )

    # cross_val_predict para matriz de confusão
    y_pred = cross_val_predict(pipe, X, y, cv=cv, n_jobs=N_JOBS, method='predict')

    f1_mean = np.mean(cv_results['test_f1_macro'])
    f1_std = np.std(cv_results['test_f1_macro'])

    print(f"\nModelo: {nome}")
    print(f"F1-macro (média ± std): {f1_mean:.4f} ± {f1_std:.4f}")

    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f'Matriz de Confusão - {nome}')
    plt.show()

# Modelos com hiperparâmetros básicos e random_state fixo
rf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=N_JOBS)
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=RANDOM_STATE)

for nome, est in [
    ('Random Forest', rf),
    ('KNN', knn),
    ('MLP', mlp)
]:
    avaliar_modelo(nome, est)
